### Preprocessing the input data

In [1]:
import numpy as np
import tensorflow as tf
import keras

batch_size = 16
img_height = 224
img_width = 224

data_dir = '../data/train'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 56358 files belonging to 5 classes.
Using 50723 files for training.
Found 56358 files belonging to 5 classes.
Using 5635 files for validation.


### MobileNetV2

In [2]:
# Downloading the model
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator

model = MobileNetV2(weights='imagenet')

### Customizing the model

In [3]:
from keras import Model
from keras.layers import Dense
import keras


EPOCHS = 300
PATIENCE = 30

corn_output = Dense(5, activation='softmax')
corn_output = corn_output(model.layers[-2].output)
corn_input = model.input

# with strategy.scope():
    
    
corn_model = Model(inputs=corn_input, outputs=corn_output)

for layer in corn_model.layers[:-1]:
    layer.trainable

    
corn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='accuracy',
        patience=PATIENCE,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoint_mobilenetv2_300.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    tf.keras.callbacks.CSVLogger(
        "Log", 
        separator=",", 
        append=False
    )
]

corn_model.fit(
    train_ds,
    steps_per_epoch=169,
    epochs=EPOCHS,
    verbose=2,
    validation_data=val_ds,
    callbacks=callbacks_list
)

model.save('mobilenetv2_300.h5')

Epoch 1/300
169/169 - 22s - loss: 0.1962 - accuracy: 0.9652 - val_loss: 0.8662 - val_accuracy: 0.9721


/home/arthur/anaconda3/envs/corn-cnn-env/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/300
169/169 - 18s - loss: 0.1351 - accuracy: 0.9763 - val_loss: 0.8766 - val_accuracy: 0.9721
Epoch 3/300
169/169 - 18s - loss: 0.1585 - accuracy: 0.9700 - val_loss: 0.5739 - val_accuracy: 0.9721
Epoch 4/300


KeyboardInterrupt: 